In [33]:
from __future__ import division
import os, warnings, tensorflow as tf
from utils import *
import numpy as np

In [34]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

## [Warning]

Do not run this notebook unless you wish to retrain the models.  
This repository is already preloaded with pre-trained VAEs for your use.

## Load datasets

In [35]:
from dsl.fashionmnist_dsl import FashionMNISTDSL

In [36]:
partition_seed = None

In [37]:
train_dsl = FashionMNISTDSL(batch_size=100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666)

if partition_seed is not None:
    np.random.seed(partition_seed)
    chosen_classes = list(np.random.choice(np.arange(train_dsl.num_classes), train_dsl.num_classes//2, False))
    
else:
    chosen_classes = list(np.arange(train_dsl.num_classes))

Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)


### Partition classes

In [38]:
train_dsl = FashionMNISTDSL(batch_size = 100, mode='train', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)
val_dsl   = FashionMNISTDSL(batch_size = 100, mode='val', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)
test_dsl  = FashionMNISTDSL(batch_size = 100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666, keep_class=chosen_classes)

Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading train data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...
Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading val data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
processing...
Creating DSL only for classes 0 1 2 3 4 5 6 7 8 9
Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)
processing...


### Obtain dataset parameters

In [39]:
width, height, channels = train_dsl.get_sample_shape()

is_multilabel = train_dsl.is_multilabel()
num_classes = train_dsl.get_num_classes()
num_batches = train_dsl.get_num_batches()

In [40]:
!pip install tensorflow-determinism==0.3.0

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


## Create model

In [41]:
import tensorflow as tf
from models.deepcnn import DeepCNN
from models.vae import NewHSVNVAE

In [42]:
from cfg import cfg, config

In [43]:
with tf.variable_scope("vae", reuse=tf.AUTO_REUSE):
    vae = NewHSVNVAE(
            batch_size=cfg.batch_size,
            height=height, width=width,
            channels=channels,
            num_classes=2,
            is_training=True,
            z_size=32,
            random_draws=1,
            noise_mean=5.0
         )

Tensor("vae_1/Decoder/conv2d_transpose/LeakyRelu:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_1/LeakyRelu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_2/LeakyRelu:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_3/LeakyRelu:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("vae_1/Decoder/conv2d_transpose_4/Sigmoid:0", shape=(?, 28, 28, 1), dtype=float32)
Adam Learning rate: 0.001


In [44]:
saver = tf.train.Saver()

In [45]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

## Create Training Dataset

In [46]:
import copy

In [47]:
gaussian_noise_added_train_dsl = copy.deepcopy(train_dsl)
    
num_train_samples = gaussian_noise_added_train_dsl.data.shape[0]

locs = np.random.uniform(0.0, 1.0, size=num_train_samples)
noise_generated = np.concatenate([
                                    [
                                        np.random.normal(
                                            loc=loc,
                                            scale=0.5,
                                            size=gaussian_noise_added_train_dsl.data.shape[1:]
                                        )
                                    
                                        for loc in locs
                                    ]
                                 ])

vnoise   = np.random.uniform(low=0.2, high=0.3,  size=num_train_samples )
vnoise   = vnoise.reshape(-1, 1, 1, 1)

gaussian_noise_added_train_dsl.data = np.clip(
                                                vnoise * noise_generated
                                                + (1-vnoise) * gaussian_noise_added_train_dsl.data,
                                      0.0, 1.0) 
            
full_data = np.concatenate([
                train_dsl.data,
                gaussian_noise_added_train_dsl.data,
                #cifar_noise_train_dsl.data[:1000]
            ], axis=0)

full_labels = np.concatenate([
                np.zeros(len(train_dsl.data), dtype=np.int32),
                np.ones(len(gaussian_noise_added_train_dsl.data), dtype=np.int32)
              ], axis=0)

In [50]:
# print(full_data)

In [21]:
mod_train_dsl             = copy.deepcopy(train_dsl)

mod_train_dsl.shuffle_each_epoch = True

mod_train_dsl.data        = full_data
mod_train_dsl.labels      = full_labels
mod_train_dsl.num_classes = 2

mod_train_dsl.shuffle_data()

In [22]:
mod_val_dsl = copy.deepcopy(val_dsl)

mod_val_dsl.labels = np.zeros(len(val_dsl.data), dtype=np.int32)

mod_val_dsl.num_classes = 2

Change labels for the main datasets

In [23]:
train_dsl.labels = np.zeros_like(train_dsl.labels, dtype=np.int32)
train_dsl.num_classes = 2

In [24]:
val_dsl.labels = np.zeros_like(val_dsl.labels, dtype=np.int32)
val_dsl.num_classes = 2

## Training Loop

In [25]:
import time

In [26]:
if partition_seed is None:
    logdir_defender = 'modeldir/fashionmnist-full-custom'
else:
    logdir_defender = 'modeldir/fashionmnist-{}-custom'

In [27]:
num_batches_tr  = train_dsl.get_num_batches()
num_batches_val = val_dsl.get_num_batches()
num_samples_val = val_dsl.get_num_samples()

In [32]:
from utils.model import compute_evaluation_measure
curr_loss = None
best_loss = None
no_improvement = 0

for epoch in range(1, 101):
    epoch_time = time.time()
    t_loss     = []
    r_loss     = []
    l_loss     = []
    
    for b_tr in range(num_batches_tr): 

        trX, trY = mod_train_dsl.load_next_batch(b_tr)

        global_step, _, summary_str, loss, recon_loss, latent_loss = sess.run([
                                         vae.global_step,
                                         vae.train_op,
                                         vae.train_summary,
                                         vae.mean_loss,
                                         vae.mean_recon_loss,
                                         vae.mean_latent_loss
                                      ],
                                      feed_dict={
                                          vae.X: trX,
                                          vae.dropout_keep_prob: cfg.dropout_keep_prob,
                                          vae.labels: trY
                                      })
        t_loss.append(loss)    
        r_loss.append(recon_loss)
        l_loss.append(latent_loss)

    curr_loss = compute_evaluation_measure(vae, sess, mod_train_dsl, vae.sum_loss)

    if epoch % 10 == 0:
        valX, _ = val_dsl.load_next_batch(1)
        decoded = sess.run( vae.decoded, { vae.X: valX  } )

        artificial_image = sess.run(vae.x_hat, feed_dict={vae.z: np.random.normal(0, 1, (144, vae.z_size))})

        save_path = saver.save(sess, logdir_defender + '/model_epoch_%d' % (epoch) )       
        print "Model saved in path: %s" % save_path            


    print "Epoch: {}\tVal:{}\tTrain: {}\tRecon: {}\tLatent: {}" .format(
        '%3d' % epoch,
        curr_loss,
        np.mean(t_loss),
        np.mean(r_loss),
        np.mean(l_loss)
    ) 

Epoch:   1	Val:23.9523988902	Train: 24.012090683	Recon: 19.5279560089	Latent: 8.96827030182
Epoch:   2	Val:23.6011341527	Train: 23.583322525	Recon: 18.987077713	Latent: 9.1924943924
Epoch:   3	Val:23.1739163284	Train: 23.2642173767	Recon: 18.5909290314	Latent: 9.34657764435
Epoch:   4	Val:23.194811676	Train: 22.9571781158	Recon: 18.213809967	Latent: 9.48673534393
Epoch:   5	Val:22.5607093379	Train: 22.7512893677	Recon: 17.9573726654	Latent: 9.58783054352
Epoch:   6	Val:22.4550865326	Train: 22.5814704895	Recon: 17.750919342	Latent: 9.66109848022
Epoch:   7	Val:22.2500413882	Train: 22.4573860168	Recon: 17.5954017639	Latent: 9.72396659851
Epoch:   8	Val:22.2355148087	Train: 22.3618602753	Recon: 17.4788093567	Latent: 9.76610660553
Epoch:   9	Val:22.1315540797	Train: 22.2030830383	Recon: 17.2885494232	Latent: 9.82906627655
Model saved in path: modeldir/fashionmnist-full-custom/model_epoch_10
Epoch:  10	Val:22.3269139404	Train: 22.1513690948	Recon: 17.1939620972	Latent: 9.91481399536
Epoch: 

Epoch:  84	Val:20.5448086917	Train: 20.6162624359	Recon: 15.2628011703	Latent: 10.7069244385
Epoch:  85	Val:20.6280062701	Train: 20.5464305878	Recon: 15.2017126083	Latent: 10.6894340515
Epoch:  86	Val:20.4822881749	Train: 20.5525913239	Recon: 15.2099647522	Latent: 10.6852579117
Epoch:  87	Val:20.5012326228	Train: 20.5854988098	Recon: 15.2411556244	Latent: 10.6886863708
Epoch:  88	Val:20.6883443527	Train: 20.6015968323	Recon: 15.2359619141	Latent: 10.7312679291
Epoch:  89	Val:20.5462960052	Train: 20.5255794525	Recon: 15.1708192825	Latent: 10.709523201
Model saved in path: modeldir/fashionmnist-full-custom/model_epoch_90
Epoch:  90	Val:20.4998600845	Train: 20.5251541138	Recon: 15.1750507355	Latent: 10.7002086639
Epoch:  91	Val:20.8176811091	Train: 20.5454463959	Recon: 15.1849889755	Latent: 10.7209129333
Epoch:  92	Val:20.4249542554	Train: 20.5650539398	Recon: 15.206530571	Latent: 10.7170495987
Epoch:  93	Val:20.6568230553	Train: 20.5383167267	Recon: 15.1955461502	Latent: 10.6855382919
Ep